In [1]:
%pylab inline

import keras.backend as K
from keras import Input, Model, Sequential
from keras.layers import *
from keras.optimizers import Adam, RMSprop
from keras.callbacks import *
from keras.engine import InputSpec, Layer
from keras import initializers, regularizers, constraints
from keras.models import load_model

import tensorflow as tf
from matplotlib import pyplot as plt

from datetime import datetime
from pathlib import Path

import pickle

from GAN_utils import *

Populating the interactive namespace from numpy and matplotlib


/home/luca/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [19]:
class WGAN:
    def __init__(self, timesteps, latent_dim, run_dir, img_dir, model_dir, generated_datesets_dir):
        self._timesteps = timesteps
        self._latent_dim = latent_dim
        self._run_dir = run_dir
        self._img_dir = img_dir
        self._model_dir = model_dir
        self._generated_datesets_dir = generated_datesets_dir
        
        self._save_config()
        
        self._epoch = 0
        self._losses = [[], []]

    def build_models(self, generator_lr, critic_lr):        
        self._generator = self._build_generator(self._latent_dim, self._timesteps)

        self._critic = self._build_critic(self._timesteps)
        self._critic.compile(loss=self._wasserstein_loss, optimizer=RMSprop(critic_lr))

        z = Input(shape=(self._latent_dim, ))
        fake = self._generator(z)

        set_model_trainable(self._critic, False)

        valid = self._critic(fake)

        self._gan = Model(z, valid, 'GAN')

        self._gan.compile(
            loss=self._wasserstein_loss,
            optimizer=RMSprop(generator_lr),
            metrics=['accuracy'])
        
        return self._gan, self._generator, self._critic

    def _build_generator(self, noise_dim, timesteps):
        generator_inputs = Input((latent_dim, ))
        generated = generator_inputs
        
        generated = Lambda(lambda x: K.expand_dims(x))(generated)
        while generated.shape[1] < timesteps:
            generated = Conv1D(
                32, 3, activation='relu', padding='same')(generated)
            generated = UpSampling1D(2)(generated)
        generated = Conv1D(
            1, 3, activation='relu', padding='same')(generated)
        generated = Lambda(lambda x: K.squeeze(x, -1))(generated)
        generated = Dense(timesteps, activation='tanh')(generated)

        generator = Model(generator_inputs, generated, 'generator')
        return generator

    def _build_critic(self, timesteps):
        critic_inputs = Input((timesteps, ))
        criticized = critic_inputs
        
#         mbd = MinibatchDiscrimination(5, 3)(criticized)
#         mbd = Dense(1, activation='tanh')(mbd)
        
        criticized = Lambda(lambda x: K.expand_dims(x))(
            criticized)
        while criticized.shape[1] > 1:
            criticized = Conv1D(
                32, 3, activation='relu', padding='same')(criticized)
            criticized = MaxPooling1D(2, padding='same')(criticized)
        criticized = Flatten()(criticized)
        criticized = Dense(32, activation='relu')(criticized)
        criticized = Dense(15, activation='relu')(criticized)
#         criticized = Dense(1, activation='tanh')(criticized) 
#         criticized = Concatenate()([criticized, mbd])
        criticized = Dense(1)(criticized) 

        critic = Model(critic_inputs, criticized, 'critic')
        return critic

    def train(self, batch_size, epochs, n_generator, n_critic, dataset, clip_value,
           img_frequency, model_save_frequency, dataset_generation_frequency, dataset_generation_size):
        half_batch = int(batch_size / 2)

        
        while self._epoch < epochs:
            self._epoch += 1
            for _ in range(n_critic):
                indexes = np.random.randint(0, dataset.shape[0], half_batch)
                batch_transactions = dataset[indexes]

                noise = np.random.normal(0, 1, (half_batch, self._latent_dim))

                generated_transactions = self._generator.predict(noise)

                critic_loss_real = self._critic.train_on_batch(
                    batch_transactions, -np.ones((half_batch, 1)))
                critic_loss_fake = self._critic.train_on_batch(
                    generated_transactions, np.ones((half_batch, 1)))
                critic_loss = 0.5 * np.add(critic_loss_real,
                                                  critic_loss_fake)

                self._clip_weights(clip_value)

            for _ in range(n_generator):
                noise = np.random.normal(0, 1, (batch_size, latent_dim))

                generator_loss = self._gan.train_on_batch(
                    noise, -np.ones((batch_size, 1)))[0]
            
            generator_loss = 1 - generator_loss
            critic_loss = 1 - critic_loss
            
            self._losses[0].append(generator_loss)
            self._losses[1].append(critic_loss)

            print("%d [C loss: %f] [G loss: %f]" % (self._epoch, critic_loss,
                                                    generator_loss))

            if self._epoch % img_frequency == 0:
                self._save_imgs()
                self._save_latent_space()
            
            if self._epoch % model_save_frequency == 0:
                self._save_models()
                
            if self._epoch % dataset_generation_frequency == 0:
                self._generate_dataset(self._epoch, dataset_generation_size)
                
            if self._epoch % 250 == 0:
                self._save_losses()
          
        self._generate_dataset(epochs, dataset_generation_size)
        self._save_losses(self._losses)
        self._save_models()
        self._save_imgs()
        self._save_latent_space()
        
        return losses
    
    def _save_imgs(self):
        rows, columns = 5, 5
        noise = np.random.normal(0, 1, (rows * columns, latent_dim))
        generated_transactions = self._generator.predict(noise)

        plt.subplots(rows, columns, figsize=(15, 5))
        k = 1
        for i in range(rows):
            for j in range(columns):
                plt.subplot(rows, columns, k)
                plt.plot(generated_transactions[k - 1])
                plt.xticks([])
                plt.yticks([])
                plt.ylim(-1, 1)
                k += 1
        plt.tight_layout()
        plt.savefig(str(self._img_dir / ('%05d.png' % self._epoch)))
        plt.savefig(str(self._img_dir / 'last.png'))
        plt.clf()
        plt.close()
        
    def _save_latent_space(self):
        if self._latent_dim > 2:
            latent_vector = np.random.normal(0, 1, latent_dim)
        plt.subplots(5, 5, figsize=(15, 5))

        for i, v_i in enumerate(np.linspace(-2, 2, 5, True)):
            for j, v_j in enumerate(np.linspace(-2, 2, 5, True)):
                if self._latent_dim > 2:
                    latent_vector[-2:] = [v_i, v_j]
                else:
                    latent_vector = np.array([v_i, v_j])
                    
                plt.subplot(5, 5, i*5+j+1)
                plt.plot(self._generator.predict(latent_vector.reshape((1, self._latent_dim))).T)
                plt.xticks([])
                plt.yticks([])
                plt.ylim(-1, 1)
        plt.tight_layout()
        plt.savefig(str(self._img_dir / ('latent_space.png')))
        plt.clf()
        plt.close()
        
    def _save_losses(self):
        plt.figure(figsize=(15, 3))
        plt.plot(self._losses[0])
        plt.plot(self._losses[1])
        plt.legend(['generator', 'critic'])
        plt.savefig(str(self._img_dir / 'losses.png'))
        plt.clf()
        plt.close()
        
        with open(str(self._run_dir / 'losses.p'), 'wb') as f:
            pickle.dump(self._losses, f)
        
    def _clip_weights(self, clip_value):
        for l in self._critic.layers:
#             if 'minibatch_discrimination' not in l.name:
            weights = [np.clip(w, -clip_value, clip_value) for w in l.get_weights()]
            l.set_weights(weights)

    def _save_config(self):
        config = {
            'timesteps' : self._timesteps,
            'latent_dim' : self._latent_dim,
            'run_dir' : self._run_dir,
            'img_dir' : self._img_dir,
            'model_dir' : self._model_dir,
            'generated_datesets_dir' : self._generated_datesets_dir
        }
        
        with open(str(self._run_dir / 'config.p'), 'wb') as f:
            pickle.dump(config, f)
        
    def _save_models(self):
        self._gan.save(self._model_dir / 'wgan.h5')
        self._generator.save(self._model_dir / 'generator.h5')
        self._critic.save(self._model_dir / 'critic.h5')
        
    def _generate_dataset(self, epoch, dataset_generation_size):
        z_samples = np.random.normal(0, 1, (dataset_generation_size, self._latent_dim))
        generated_dataset = self._generator.predict(z_samples)
        np.save(self._generated_datesets_dir / ('%d_generated_data' % epoch), generated_dataset)
        np.save(self._generated_datesets_dir / 'last', generated_dataset)
        
    def get_models(self):
        return self._gan, self._generator, self._critic
    
    @staticmethod
    def _wasserstein_loss(y_true, y_pred):
        return K.mean(y_true * y_pred)

    def restore_training(self):
        self.load_models()
        with open(str(self._run_dir / 'losses.p'), 'rb') as f:
            self._losses = pickle.load(f)
            self._epoch = len(self._losses[0])
        
        return self._gan, self._generator, self._critic
    
    def load_models(self):
        custom_objects = {
            'MinibatchDiscrimination':MinibatchDiscrimination,
            '_wasserstein_loss':self._wasserstein_loss
        }
        self._gan = load_model(self._model_dir / 'wgan.h5', custom_objects=custom_objects)
        self._generator = load_model(self._model_dir / 'generator.h5')
        self._critic = load_model(self._model_dir / 'critic.h5', custom_objects=custom_objects)
        
        return self._gan, self._generator, self._critic

In [20]:
normalized_transactions_filepath = "../../datasets/berka_dataset/usable/normalized_transactions_100.npy"

timesteps = 100
transactions = np.load(normalized_transactions_filepath)
transactions.shape

(53888, 100)

In [24]:
batch_size = 64
epochs = 500000
n_critic = 5
n_generator = 1
latent_dim = 2
generator_lr = 0.00005
critic_lr = 0.00005
clip_value = 0.05
img_frequency = 250
model_save_frequency = 3000
dataset_generation_frequency = 25000
dataset_generation_size = 100000

In [25]:
root_path = Path('wgan')
if not root_path.exists():
    root_path.mkdir()
    
current_datetime = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')

run_dir = root_path / current_datetime
img_dir = run_dir / 'img'
model_dir = run_dir / 'models'
generated_datesets_dir = run_dir / 'generated_datasets'

img_dir.mkdir(parents=True)
model_dir.mkdir(parents=True)
generated_datesets_dir.mkdir(parents=True)

In [26]:
wgan = WGAN(timesteps, latent_dim, run_dir, img_dir, model_dir, generated_datesets_dir)
# gan, generator, critic = wgan.restore_training()
gan, generator, critic = wgan.build_models(generator_lr, critic_lr)
        
losses = wgan.train(batch_size, epochs, n_generator, n_critic, transactions, clip_value,
           img_frequency, model_save_frequency, dataset_generation_frequency, dataset_generation_size)

/home/luca/.local/lib/python3.5/site-packages/keras/engine/training.py:953: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [C loss: 0.999962] [G loss: 1.000086]
2 [C loss: 0.999966] [G loss: 1.000090]
3 [C loss: 0.999967] [G loss: 1.000090]
4 [C loss: 0.999966] [G loss: 1.000089]
5 [C loss: 0.999966] [G loss: 1.000089]
6 [C loss: 0.999966] [G loss: 1.000088]
7 [C loss: 0.999966] [G loss: 1.000088]
8 [C loss: 0.999966] [G loss: 1.000088]
9 [C loss: 0.999966] [G loss: 1.000088]
10 [C loss: 0.999966] [G loss: 1.000088]
11 [C loss: 0.999966] [G loss: 1.000087]
12 [C loss: 0.999966] [G loss: 1.000087]
13 [C loss: 0.999966] [G loss: 1.000087]
14 [C loss: 0.999966] [G loss: 1.000087]
15 [C loss: 0.999966] [G loss: 1.000087]
16 [C loss: 0.999966] [G loss: 1.000087]
17 [C loss: 0.999967] [G loss: 1.000087]
18 [C loss: 0.999967] [G loss: 1.000087]
19 [C loss: 0.999967] [G loss: 1.000086]
20 [C loss: 0.999967] [G loss: 1.000086]
21 [C loss: 0.999967] [G loss: 1.000086]
22 [C loss: 0.999966] [G loss: 1.000086]
23 [C loss: 0.999967] [G loss: 1.000086]
24 [C loss: 0.999967] [G loss: 1.000086]
25 [C loss: 0.999967] [G 

199 [C loss: 1.029191] [G loss: 0.998459]
200 [C loss: 1.030733] [G loss: 0.998433]
201 [C loss: 1.031896] [G loss: 0.998406]
202 [C loss: 1.032856] [G loss: 0.998383]
203 [C loss: 1.033467] [G loss: 0.998357]
204 [C loss: 1.034318] [G loss: 0.998339]
205 [C loss: 1.035921] [G loss: 0.998314]
206 [C loss: 1.036867] [G loss: 0.998291]
207 [C loss: 1.039741] [G loss: 0.998264]
208 [C loss: 1.041164] [G loss: 0.998244]
209 [C loss: 1.042682] [G loss: 0.998214]
210 [C loss: 1.043002] [G loss: 0.998185]
211 [C loss: 1.044512] [G loss: 0.998169]
212 [C loss: 1.046557] [G loss: 0.998140]
213 [C loss: 1.046929] [G loss: 0.998112]
214 [C loss: 1.050363] [G loss: 0.998095]
215 [C loss: 1.049861] [G loss: 0.998066]
216 [C loss: 1.054217] [G loss: 0.998045]
217 [C loss: 1.053543] [G loss: 0.998015]
218 [C loss: 1.056348] [G loss: 0.997991]
219 [C loss: 1.060454] [G loss: 0.997970]
220 [C loss: 1.062093] [G loss: 0.997943]
221 [C loss: 1.063095] [G loss: 0.997919]
222 [C loss: 1.063630] [G loss: 0.

395 [C loss: 0.941371] [G loss: 3.482216]
396 [C loss: 1.029515] [G loss: 3.474463]
397 [C loss: 0.815254] [G loss: 3.529808]
398 [C loss: 0.949855] [G loss: 3.470333]
399 [C loss: 0.823610] [G loss: 3.436219]
400 [C loss: 0.869663] [G loss: 3.487194]
401 [C loss: 0.877480] [G loss: 3.451404]
402 [C loss: 0.702220] [G loss: 3.461773]
403 [C loss: 0.720172] [G loss: 3.668455]
404 [C loss: 0.693384] [G loss: 3.655111]
405 [C loss: 0.731478] [G loss: 3.645340]
406 [C loss: 0.793424] [G loss: 3.405749]
407 [C loss: 0.695722] [G loss: 3.663854]
408 [C loss: 0.788032] [G loss: 3.596620]
409 [C loss: 0.730693] [G loss: 3.721852]
410 [C loss: 0.739372] [G loss: 3.559645]
411 [C loss: 0.703223] [G loss: 3.611493]
412 [C loss: 0.761944] [G loss: 3.531432]
413 [C loss: 0.764208] [G loss: 3.554677]
414 [C loss: 0.738106] [G loss: 3.591326]
415 [C loss: 0.598332] [G loss: 3.622283]
416 [C loss: 0.724599] [G loss: 3.428625]
417 [C loss: 0.634483] [G loss: 3.479530]
418 [C loss: 0.518140] [G loss: 3.

591 [C loss: 1.014051] [G loss: 0.896692]
592 [C loss: 1.021236] [G loss: 0.897131]
593 [C loss: 1.013812] [G loss: 0.895096]
594 [C loss: 1.012182] [G loss: 0.903457]
595 [C loss: 1.013732] [G loss: 0.894433]
596 [C loss: 1.009508] [G loss: 0.895457]
597 [C loss: 1.010673] [G loss: 0.898152]
598 [C loss: 1.005520] [G loss: 0.893376]
599 [C loss: 1.003075] [G loss: 0.903993]
600 [C loss: 1.010635] [G loss: 0.896942]
601 [C loss: 0.997068] [G loss: 0.902793]
602 [C loss: 1.003100] [G loss: 0.909763]
603 [C loss: 1.002316] [G loss: 0.906233]
604 [C loss: 1.000300] [G loss: 0.913096]
605 [C loss: 1.000375] [G loss: 0.912717]
606 [C loss: 1.001908] [G loss: 0.919356]
607 [C loss: 1.001525] [G loss: 0.907248]
608 [C loss: 1.002056] [G loss: 0.922677]
609 [C loss: 0.997573] [G loss: 0.929490]
610 [C loss: 0.998074] [G loss: 0.927597]
611 [C loss: 0.997787] [G loss: 0.919768]
612 [C loss: 0.996181] [G loss: 0.930372]
613 [C loss: 0.999604] [G loss: 0.932534]
614 [C loss: 0.995387] [G loss: 0.

...

1365 [C loss: 0.999952] [G loss: 1.002098]
1366 [C loss: 0.999950] [G loss: 1.002098]
1367 [C loss: 0.999952] [G loss: 1.002095]
1368 [C loss: 0.999952] [G loss: 1.002095]
1369 [C loss: 0.999952] [G loss: 1.002092]
1370 [C loss: 0.999952] [G loss: 1.002091]
1371 [C loss: 0.999954] [G loss: 1.002091]
1372 [C loss: 0.999951] [G loss: 1.002088]
1373 [C loss: 0.999951] [G loss: 1.002088]
1374 [C loss: 0.999952] [G loss: 1.002088]
1375 [C loss: 0.999950] [G loss: 1.002086]
1376 [C loss: 0.999952] [G loss: 1.002087]
1377 [C loss: 0.999952] [G loss: 1.002086]
1378 [C loss: 0.999951] [G loss: 1.002086]
1379 [C loss: 0.999952] [G loss: 1.002085]
1380 [C loss: 0.999952] [G loss: 1.002082]
1381 [C loss: 0.999953] [G loss: 1.002083]
1382 [C loss: 0.999951] [G loss: 1.002083]
1383 [C loss: 0.999952] [G loss: 1.002082]
1384 [C loss: 0.999953] [G loss: 1.002081]
1385 [C loss: 0.999953] [G loss: 1.002079]
1386 [C loss: 0.999951] [G loss: 1.002079]
1387 [C loss: 0.999954] [G loss: 1.002078]
1388 [C los

1556 [C loss: 0.999953] [G loss: 1.002054]
1557 [C loss: 0.999954] [G loss: 1.002052]
1558 [C loss: 0.999953] [G loss: 1.002051]
1559 [C loss: 0.999953] [G loss: 1.002050]
1560 [C loss: 0.999953] [G loss: 1.002049]
1561 [C loss: 0.999953] [G loss: 1.002047]
1562 [C loss: 0.999954] [G loss: 1.002046]
1563 [C loss: 0.999953] [G loss: 1.002045]
1564 [C loss: 0.999954] [G loss: 1.002043]
1565 [C loss: 0.999953] [G loss: 1.002042]
1566 [C loss: 0.999954] [G loss: 1.002041]
1567 [C loss: 0.999954] [G loss: 1.002040]
1568 [C loss: 0.999954] [G loss: 1.002038]
1569 [C loss: 0.999954] [G loss: 1.002037]
1570 [C loss: 0.999954] [G loss: 1.002036]
1571 [C loss: 0.999954] [G loss: 1.002035]
1572 [C loss: 0.999954] [G loss: 1.002034]
1573 [C loss: 0.999954] [G loss: 1.002032]
1574 [C loss: 0.999954] [G loss: 1.002031]
1575 [C loss: 0.999954] [G loss: 1.002030]
1576 [C loss: 0.999954] [G loss: 1.002029]
1577 [C loss: 0.999954] [G loss: 1.002028]
1578 [C loss: 0.999954] [G loss: 1.002027]
1579 [C los

1747 [C loss: 0.999961] [G loss: 1.001897]
1748 [C loss: 0.999959] [G loss: 1.001899]
1749 [C loss: 0.999964] [G loss: 1.001900]
1750 [C loss: 0.999963] [G loss: 1.001900]
1751 [C loss: 0.999960] [G loss: 1.001904]
1752 [C loss: 0.999958] [G loss: 1.001904]
1753 [C loss: 0.999957] [G loss: 1.001893]
1754 [C loss: 0.999960] [G loss: 1.001894]
1755 [C loss: 0.999965] [G loss: 1.001886]
1756 [C loss: 0.999963] [G loss: 1.001882]
1757 [C loss: 0.999962] [G loss: 1.001881]
1758 [C loss: 0.999962] [G loss: 1.001881]
1759 [C loss: 0.999963] [G loss: 1.001882]
1760 [C loss: 0.999961] [G loss: 1.001885]
1761 [C loss: 0.999961] [G loss: 1.001886]
1762 [C loss: 0.999959] [G loss: 1.001890]
1763 [C loss: 0.999956] [G loss: 1.001893]
1764 [C loss: 0.999959] [G loss: 1.001893]
1765 [C loss: 0.999966] [G loss: 1.001894]
1766 [C loss: 0.999965] [G loss: 1.001899]
1767 [C loss: 0.999972] [G loss: 1.001909]
1768 [C loss: 0.999976] [G loss: 1.001916]
1769 [C loss: 0.999972] [G loss: 1.001950]
1770 [C los

1938 [C loss: 0.999970] [G loss: 1.001812]
1939 [C loss: 0.999963] [G loss: 1.001829]
1940 [C loss: 0.999999] [G loss: 1.001794]
1941 [C loss: 0.999961] [G loss: 1.001829]
1942 [C loss: 0.999950] [G loss: 1.001787]
1943 [C loss: 0.999958] [G loss: 1.001770]
1944 [C loss: 0.999969] [G loss: 1.001766]
1945 [C loss: 0.999971] [G loss: 1.001749]
1946 [C loss: 0.999979] [G loss: 1.001688]
1947 [C loss: 0.999997] [G loss: 1.001692]
1948 [C loss: 0.999966] [G loss: 1.001716]
1949 [C loss: 0.999978] [G loss: 1.001709]
1950 [C loss: 0.999971] [G loss: 1.001730]
1951 [C loss: 0.999960] [G loss: 1.001728]
1952 [C loss: 0.999967] [G loss: 1.001718]
1953 [C loss: 0.999970] [G loss: 1.001716]
1954 [C loss: 0.999959] [G loss: 1.001705]
1955 [C loss: 0.999975] [G loss: 1.001707]
1956 [C loss: 0.999988] [G loss: 1.001679]
1957 [C loss: 0.999976] [G loss: 1.001677]
1958 [C loss: 0.999984] [G loss: 1.001691]
1959 [C loss: 0.999976] [G loss: 1.001694]
1960 [C loss: 0.999974] [G loss: 1.001658]
1961 [C los

KeyboardInterrupt: 